In [1]:
import time
import requests
from bs4 import BeautifulSoup
from plyer import notification
import tkinter as tk
from tkinter import simpledialog, messagebox

def get_latest_news(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        first_news = soup.find("h2")  # Örnek olarak ilk haber başlığını alıyoruz
        return first_news.text.strip() if first_news else None
    except Exception as e:
        print(f"Hata oluştu: {e}")
        return None

def check_news():
    global tracked_sites, last_news
    while True:
        for url in tracked_sites:
            latest_news = get_latest_news(url)
            if latest_news and latest_news != last_news.get(url, None):
                notification.notify(
                    title="Yeni Haber Var!",
                    message=f"{latest_news}\nKaynak: {url}",
                    timeout=10
                )
                last_news[url] = latest_news
        time.sleep(5)

def add_site():
    global tracked_sites
    url = simpledialog.askstring("Site Ekle", "Lütfen haber sitesinin URL'sini girin:")
    if url:
        tracked_sites.append(url)
        listbox.insert(tk.END, url)

def remove_site():
    global tracked_sites
    selected = listbox.curselection()
    if selected:
        index = selected[0]
        url = listbox.get(index)
        tracked_sites.remove(url)
        listbox.delete(index)
    else:
        messagebox.showwarning("Uyarı", "Silmek için bir site seçin!")

def start_tracking():
    messagebox.showinfo("Başlat", "Haber takibi başlatıldı! Arka planda çalışıyor.")
    check_news()

# UI oluşturma
root = tk.Tk()
root.title("Haber Takip Uygulaması")

tracked_sites = []
last_news = {}

frame = tk.Frame(root)
frame.pack(pady=10)

listbox = tk.Listbox(frame, width=50, height=10)
listbox.pack()

btn_add = tk.Button(root, text="Site Ekle", command=add_site)
btn_add.pack(pady=5)

btn_remove = tk.Button(root, text="Site Kaldır", command=remove_site)
btn_remove.pack(pady=5)

btn_start = tk.Button(root, text="Takibi Başlat", command=start_tracking)
btn_start.pack(pady=10)

root.mainloop()


Hata oluştu: HTTPSConnectionPool(host='www.haberturk.com', port=443): Max retries exceeded with url: /gundem (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022FD152BF20>: Failed to resolve 'www.haberturk.com' ([Errno 11001] getaddrinfo failed)"))
Hata oluştu: HTTPSConnectionPool(host='www.haberturk.com', port=443): Max retries exceeded with url: /gundem (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022FD140BE00>: Failed to resolve 'www.haberturk.com' ([Errno 11001] getaddrinfo failed)"))
Hata oluştu: HTTPSConnectionPool(host='www.haberturk.com', port=443): Max retries exceeded with url: /gundem (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000022FD19F1EE0>: Failed to resolve 'www.haberturk.com' ([Errno 11001] getaddrinfo failed)"))


: 

In [ ]:
import time
import requests
from bs4 import BeautifulSoup
from plyer import notification
import tkinter as tk
from tkinter import simpledialog, messagebox
import threading

def get_latest_news(url):
    """
    İlgili URL'den haber başlığı çekmeye çalışır.
    Hatalarda None döner.
    """
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Örnek: h2 etiketli ilk haber başlığı
        first_news = soup.find("h2")
        return first_news.text.strip() if first_news else None

    except requests.exceptions.Timeout:
        print(f"Zaman aşımı: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Hata oluştu: {e}")
        return None

def check_news_loop():
    """
    Ayrı bir thread içinde çalışan döngü.
    Her 30 saniyede bir listedeki siteleri kontrol eder.
    """
    global running

    while running:  # 'running' True olduğu sürece çalışır
        for url in tracked_sites:
            latest_news = get_latest_news(url)
            if latest_news and latest_news != last_news.get(url, None):
                notification.notify(
                    title="Yeni Haber Var!",
                    message=f"{latest_news}\nKaynak: {url}",
                    timeout=10
                )
                last_news[url] = latest_news
        # 30 saniye bekle, ancak UI bloklanmasın diye normal sleep
        # thread içinde olduğu için sorun yaratmaz
        time.sleep(30)

def add_site():
    """
    Kullanıcıdan bir URL ister ve listeye ekler.
    Aynı site birden fazla kez eklenmek istenirse uyarı verir.
    """
    url = simpledialog.askstring("Site Ekle", "Lütfen haber sitesinin URL'sini girin:")
    if url:
        # Aynı sitenin eklenmesini engelle
        if url in tracked_sites:
            messagebox.showinfo("Bilgi", "Bu site zaten ekli.")
        else:
            tracked_sites.append(url)
            listbox.insert(tk.END, url)

def remove_site():
    """
    Seçili olan siteyi listeden çıkarır.
    """
    selected = listbox.curselection()
    if selected:
        index = selected[0]
        url = listbox.get(index)
        tracked_sites.remove(url)
        listbox.delete(index)
        # last_news sözlüğünden de sil ki tekrar eklendiğinde sıfırlansın
        if url in last_news:
            del last_news[url]
    else:
        messagebox.showwarning("Uyarı", "Silmek için bir site seçin!")

def start_tracking():
    """
    Takibi başlatır ve kullanıcıya bilgi verir.
    Arka planda çalışan thread'i başlatır.
    """
    global running, check_thread
    if running:
        messagebox.showinfo("Bilgi", "Takip zaten başlatılmış.")
        return

    running = True
    check_thread = threading.Thread(target=check_news_loop, daemon=True)
    check_thread.start()
    messagebox.showinfo("Başlat", "Haber takibi başlatıldı! Arka planda çalışıyor.")

def stop_tracking():
    """
    Takibi durdurur.
    Thread'in while döngüsünü bitirmek için 'running' değişkenini False yapar.
    """
    global running
    if running:
        running = False
        messagebox.showinfo("Durdur", "Haber takibi durduruldu.")
    else:
        messagebox.showinfo("Bilgi", "Takip zaten durdurulmuş.")

def on_closing():
    """
    Pencereyi kapatma isteği geldiğinde çalışan fonksiyon.
    Programın arka planda koşan thread'ini sonlandırır.
    """
    if messagebox.askokcancel("Çıkış", "Uygulamadan çıkmak istediğinize emin misiniz?"):
        stop_tracking()  # Thread sonlandırma sinyali
        root.destroy()

# Global değişkenler
tracked_sites = []
last_news = {}

running = False  # Thread'i kontrol etmek için
check_thread = None

# UI oluşturma
root = tk.Tk()
root.title("Haber Takip Uygulaması")

frame = tk.Frame(root)
frame.pack(pady=10)

listbox = tk.Listbox(frame, width=50, height=10)
listbox.pack()

btn_add = tk.Button(root, text="Site Ekle", command=add_site)
btn_add.pack(pady=5)

btn_remove = tk.Button(root, text="Site Kaldır", command=remove_site)
btn_remove.pack(pady=5)

btn_start = tk.Button(root, text="Takibi Başlat", command=start_tracking)
btn_start.pack(pady=5)

btn_stop = tk.Button(root, text="Takibi Durdur", command=stop_tracking)
btn_stop.pack(pady=5)

# Pencere kapatılınca on_closing fonksiyonunu çağır
root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()

burda doğrudan borsa takibi işi var

In [3]:
pip install ccxt

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.6 MB 5.6 MB/s eta 0:00:01
   ---------------- ----------------------- 2.4/5.6 MB 5.8 MB/s eta 0:00:01
   ------------------------- -------------- 3.7/5.6 MB 5.9 MB/s eta 0:00:01
   --------------------------------- ------ 4.7/5.6 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------- ----------------------- 1.3/3.2 MB 7.4 MB/s eta 0:00:01
   -------------------------- ------------- 2.1/3.2 MB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\Yusuf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import tkinter as tk
import threading
import websocket
import json

class BinanceWebSocketTracker(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Binance WebSocket Kripto Takibi")

        # Binance WebSocket adresi: BTC/USDT için ticker
        self.ws_url = "wss://stream.binance.com:9443/ws/btcusdt@ticker"

        self.label_text = tk.StringVar()
        self.label_text.set("Veri bekleniyor...")
        self.label = tk.Label(self, textvariable=self.label_text, font=("Arial", 14))
        self.label.pack(pady=20)

        # WebSocket başlat
        self.running = True
        self.ws = None
        threading.Thread(target=self.start_websocket, daemon=True).start()

    def on_message(self, ws, message):
        data = json.loads(message)
        # data içinde 'c' = son fiyat (lastPrice)
        last_price = data.get('c', 'N/A')
        self.label_text.set(f"BTC/USDT Fiyat: {last_price}")

    def on_error(self, ws, error):
        self.label_text.set(f"Hata: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        self.label_text.set("Bağlantı kapatıldı.")

    def on_open(self, ws):
        print("WebSocket bağlantısı kuruldu.")

    def start_websocket(self):
        # WebSocket bağlanma fonksiyonu
        self.ws = websocket.WebSocketApp(
            self.ws_url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )
        self.ws.run_forever()

    def on_closing(self):
        # Pencere kapanırken WebSocket'i de kapatalım
        self.running = False
        if self.ws:
            self.ws.close()
        self.destroy()

if __name__ == "__main__":
    app = BinanceWebSocketTracker()
    app.protocol("WM_DELETE_WINDOW", app.on_closing)
    app.mainloop()


WebSocket bağlantısı kuruldu.


In [1]:
import queue
import ccxt
import time
import threading
import tkinter as tk
from tkinter import messagebox, simpledialog

class CryptoTracker(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("CCXT Borsa Takibi - Thread Safe")
        self.markets = []

        self.market_listbox = tk.Listbox(self, width=40, height=10)
        self.market_listbox.pack(pady=10)

        self.add_button = tk.Button(self, text="Piyasa Ekle", command=self.add_market)
        self.add_button.pack(pady=5)

        self.remove_button = tk.Button(self, text="Seçili Piyasayı Kaldır", command=self.remove_market)
        self.remove_button.pack(pady=5)

        self.prices_text = tk.StringVar()
        self.prices_text.set("Henüz veri yok...")
        self.prices_label = tk.Label(self, textvariable=self.prices_text, font=("Arial", 12), justify=tk.LEFT)
        self.prices_label.pack(pady=10)

        # Thread-safety için veriyi taşımak adına bir Queue kullanıyoruz
        self.data_queue = queue.Queue()

        # Arka planda veri çekme thread'i
        self.running = True
        self.fetch_thread = threading.Thread(target=self.fetch_data_loop, daemon=True)
        self.fetch_thread.start()

        # Ana thread’de periyodik olarak UI güncelle
        self.update_ui()

    def add_market(self):
        exchange_id = simpledialog.askstring("Borsa Ekle", "Borsa ismi (ör. binance, kraken, kucoin):")
        if not exchange_id:
            return
        symbol = simpledialog.askstring("Sembol Ekle", "Sembol (ör. BTC/USDT, ETH/USDT):")
        if not symbol:
            return
        self.markets.append((exchange_id.lower().strip(), symbol.upper().strip()))
        self.market_listbox.insert(tk.END, f"{exchange_id} - {symbol}")

    def remove_market(self):
        index = self.market_listbox.curselection()
        if not index:
            messagebox.showwarning("Uyarı", "Kaldırmak için bir piyasa seçin!")
            return
        idx = index[0]
        self.markets.pop(idx)
        self.market_listbox.delete(idx)

    def fetch_data_loop(self):
        """
        Arka planda çalışan thread, sürekli (örnek: 5 sn'de bir) veri çeker
        ve data_queue'ya gönderir. UI güncellemesi bu thread'de yapılmaz.
        """
        while self.running:
            lines = []
            for exchange_id, symbol in self.markets:
                try:
                    exchange_class = getattr(ccxt, exchange_id)
                    exchange = exchange_class()
                    ticker = exchange.fetch_ticker(symbol)
                    last_price = ticker.get('last', None)
                    if last_price is not None:
                        lines.append(f"{exchange_id} - {symbol}: {last_price}")
                    else:
                        lines.append(f"{exchange_id} - {symbol}: Fiyat alınamadı!")
                except Exception as e:
                    lines.append(f"{exchange_id} - {symbol} Hata: {e}")

            if lines:
                # veriyi UI thread'ine aktarmak için queue'ya gönder
                self.data_queue.put("\n".join(lines))
            else:
                self.data_queue.put("Liste boş. Lütfen piyasa ekleyin.")

            time.sleep(5)

    def update_ui(self):
        """
        Ana thread’de periyodik olarak queue'yu kontrol eder
        ve prices_text’i günceller.
        """
        try:
            # queue'da veri varsa alalım
            new_text = self.data_queue.get_nowait()
            self.prices_text.set(new_text)
        except queue.Empty:
            pass  # queue boşsa bekle

        if self.running:
            # 500 ms sonra (0.5 sn) tekrar update_ui çağır
            self.after(500, self.update_ui)

    def on_closing(self):
        self.running = False
        self.destroy()

if __name__ == "__main__":
    app = CryptoTracker()
    app.protocol("WM_DELETE_WINDOW", app.on_closing)
    app.mainloop()


In [ ]:
import ccxt

binance = ccxt.binance()
markets = binance.load_markets()

# markets bir sözlük (dict) döndürür, anahtarlar 'BTC/USDT' gibi sembollerdir
print("Binance'te aktif tüm türk lirası işlem çiftleri:")
for symbol in markets.keys():
    if "TRY" in symbol:
        print(symbol)


Binance'te aktif tüm işlem çiftleri:
BTC/TRY
BNB/TRY
BUSD/TRY
ETH/TRY
XRP/TRY
USDT/TRY
LINK/TRY
SXP/TRY
AVAX/TRY
TRX/TRY
CHZ/TRY
XLM/TRY
DOGE/TRY
DOT/TRY
ADA/TRY
HOT/TRY
BTT/TRY
NEO/TRY
EOS/TRY
RVN/TRY
ONT/TRY
VET/TRY
SHIB/TRY
MATIC/TRY
SOL/TRY
GRT/TRY
TLM/TRY
ARPA/TRY
LAZIO/TRY
REEF/TRY
MANA/TRY
SAND/TRY
PORTO/TRY
SLP/TRY
FIS/TRY
LRC/TRY
SANTOS/TRY
BEL/TRY
DENT/TRY
ENJ/TRY
ALICE/TRY
GALA/TRY
LUNA/TRY
FTM/TRY
MINA/TRY
XTZ/TRY
COCOS/TRY
ATOM/TRY
ICP/TRY
ROSE/TRY
BTTC/TRY
COS/TRY
ONE/TRY
SPELL/TRY
AXS/TRY
DAR/TRY
NEAR/TRY
ALPINE/TRY
INJ/TRY
API3/TRY
MBOX/TRY
UMA/TRY
TWT/TRY
APE/TRY
JASMY/TRY
FIL/TRY
ZIL/TRY
GMT/TRY
WAVES/TRY
BSW/TRY
AUDIO/TRY
GAL/TRY
ENS/TRY
ALGO/TRY
RUNE/TRY
STORJ/TRY
ETC/TRY
ANKR/TRY
APT/TRY
FET/TRY
CFX/TRY
STX/TRY
AGIX/TRY
ARB/TRY
ID/TRY
JOE/TRY
MAGIC/TRY
ACH/TRY
XVS/TRY
EDU/TRY
SUI/TRY
RNDR/TRY
OG/TRY
PEPE/TRY
FLOKI/TRY
CITY/TRY
COMBO/TRY
LTC/TRY
RAD/TRY
OP/TRY
PAXG/TRY
MAV/TRY
OCEAN/TRY
TUSD/TRY
BCH/TRY
XVG/TRY
ARKM/TRY
ACA/TRY
COMP/TRY
XEC/TRY
WLD/TRY
AMP/TRY
OGN/T